# Notebook to optimize CLDM parameter against the nuclear mass table. With or without surface modification asurf(eCLDM)

## Import python packages


In [1]:
import numpy as np

import os
import sys


# Define the path for the shared object

In [2]:
nseospy_tk = os.getenv('NSEOSPY_TK')
sys.path.insert(0, nseospy_tk)

# Import from nucleus python data from AMDC and routine to fit finite size coefficients

In [4]:
from nseospy import read_amdc
from nseospy import nucleus_cldm_fit_coef_FS

10


## Define boundaris of data table you want to fit. Amin, Zmin
## Chose the use of interpolated data, yes ='y' or no='n'
## Chose version of the release table. Options are: 2012, 2016 and 2020.

In [5]:
Amin = 12
Zmin = 6
interpolated = 'n'
version='2012'

# Read AMDC mass table.
## The output is a big table that shows all nuclei that will be use in the fit.

In [6]:
    print('-'*20)
    print("Read AMDC: run over all table")
    print('-'*20)
    #
    GsNucData = read_amdc( Amin=Amin, Zmin=Zmin, interp=interpolated, version=version )
    #
    print("   Print output:")
    print("Mass.version:",GsNucData.version)
    print("Mass.interp :",GsNucData.interp)
    print("Mass.nbLine :",GsNucData.nbLine)
    print("Mass.nbNuc  :",GsNucData.nbNuc)

--------------------
Read AMDC: run over all table
--------------------
   Print output:
Mass.version: 2012
Mass.interp : n
Mass.nbLine : 5513
Mass.nbNuc  : 2384



  # Define FS model, FS1, FS2, FS3, or FS4
  ### FS1 = contains only surface and direct Coulomb terms. The density of the nucleus is considered constant and equal to the saturation density.
  ### FS2 = contains only surface and direct Coulomb terms. The density of the nucleus is optimized by the mechanical equilibrum of the nucleus, i.e., Pnuc = 0.
  ### FS3 = same as FS2, with the addition of curvature term.
  ### FS4 = same as FS3, with the addition of exchange Coulomb term.
  
  ## FS1 and FS2 optimize 3 parameters while FS3 and FS4 optimize 5.
  

In [8]:
FSmodel = 4

# Define meta-model parameters.

## You can chose one from the dictionary for options, or define yourself the nuclear empirical parameters you want.

## In the present example we have BSK24, from Brussels Skyrme functionals, ref:

In [10]:
MMparam='BSK24opt2'
# MMparam='SLy5sat'
# asurf=0 : standard CLDM
asurf=0.0
# asurf=-20.0 : extendended CLDM, contains densisity dependence on the surface tension.
# asurf=-20.0

  # With all defined now we fit finite size (FS) terms.
  ## We use the well-known Gauss-Newton algorithm, which converges fast.
  
  ### The fit presciption is explain with detailns in arXiv: 2110.00441

In [11]:
    # compute for different psurf
    psurf = [2.9,3.1,3.3,3.5]
    #
    std_sigSurfSat = 1.1
    std_sigSurfSym = 2.3
    std_bSurf = 29.9
    std_alphaCurv = 5.5
    std_betaCurv = 0.7
    std_sigCurv = 0.1
    #    
    print('-'*20)
    print("Call nucleus_cldm_fit_coef_FS:")
    print('-'*20)
    #FSmodel = 3
    p = 3.1
    mod = [1,2,3]
#    for p in psurf:    
    for FSmodel in mod:
        fs_param_std = np.array([ std_sigSurfSat, std_sigSurfSym,std_bSurf, p, std_alphaCurv, std_betaCurv, std_sigCurv, -12.0, 44.0, 2.0], dtype=np.float64) 
        nucfit_fs = nucleus_cldm_fit_coef_FS( GsNucData, aFsFlag = np.array([FSmodel, 1, 0 ], dtype=np.int32), mf_model="mm_", mm_param=MMparam, fs_param = "BSK14", fs_param_std =fs_param_std, asurf =asurf  )
        #
        print("   Print output:")
        print(" FSmodel: ",FSmodel)
        print("   psurf: ",p)
        print("   coef: ",nucfit_fs.afsParam)
        print("   asurf:",nucfit_fs.asurfParam)
        print(" (chi2)^1/2 :",nucfit_fs.chi_Etot)
        #
        coefFS = nucfit_fs.afsParam
        print('final parameters :')
        print("c_coul                : ",coefFS[0])
        print("c_surf_sat*sigSurfsat : ",coefFS[1]*std_sigSurfSat)
        print("c_surf_sym*sigSurfsym : ",coefFS[2]*std_sigSurfSym)
        print("c_curv*sigCurv        : ",coefFS[3]*std_sigCurv)
        print("c_beta*betaCurv       : ",coefFS[4]*std_betaCurv)
        #exit()    


--------------------
Call nucleus_cldm_fit_coef_FS:
--------------------
   Print output:
 FSmodel:  1
   psurf:  3.1
   coef:  [ 0.96545802  1.03346376  0.61793964 -6.40271227 -5.16657224  1.
  1.          1.        ]
   asurf: 0.0
 (chi2)^1/2 : 3.2657953269976283
final parameters :
c_coul                :  0.9654580205242845
c_surf_sat*sigSurfsat :  1.1368101342694137
c_surf_sym*sigSurfsym :  1.421261167801923
c_curv*sigCurv        :  -0.6402712267073035
c_beta*betaCurv       :  -3.616600568432603
   Print output:
 FSmodel:  2
   psurf:  3.1
   coef:  [ 0.95512068  1.05339391  0.70950074 -1.02608442 -0.23476882  1.
  1.          1.        ]
   asurf: 0.0
 (chi2)^1/2 : 2.9155497658385685
final parameters :
c_coul                :  0.9551206779534274
c_surf_sat*sigSurfsat :  1.1587332970105015
c_surf_sym*sigSurfsym :  1.6318516980000448
c_curv*sigCurv        :  -0.10260844220974477
c_beta*betaCurv       :  -0.16433817678822207
   Print output:
 FSmodel:  3
   psurf:  3.1
   coef:  [ 0.

## Print in screen the output for the optimized coefficients:

In [12]:
print("c_coul: ",coefFS[0],"c_surf_sat: ",coefFS[1],"c_surf_sym: ",coefFS[2],"c_curv: ",coefFS[3],"c_beta: ",coefFS[4])

print()
print('psurf = : ', psurf)
print()
print('final parameters :')
print("c_coul                : ",coefFS[0])
print("c_surf_sat*sigSurfsat : ",coefFS[1]*std_sigSurfSat)
print("c_surf_sym*sigSurfsym : ",coefFS[2]*std_sigSurfSym)
print("c_curv*sigCurv        : ",coefFS[3]*std_sigCurv)
print("c_beta*betaCurv       : ",coefFS[4]*std_betaCurv)
print()
print(" (chi2)^1/2 :",nucfit_fs.chi_Etot)

c_coul:  0.9663847830160506 c_surf_sat:  1.0122551605357595 c_surf_sym:  0.7792015995687019 c_curv:  1.04062772130783 c_beta:  0.8768274391166619

psurf = :  [2.9, 3.1, 3.3, 3.5]

final parameters :
c_coul                :  0.9663847830160506
c_surf_sat*sigSurfsat :  1.1134806765893355
c_surf_sym*sigSurfsym :  1.7921636790080144
c_curv*sigCurv        :  0.10406277213078302
c_beta*betaCurv       :  0.6137792073816634

 (chi2)^1/2 : 2.7332540785539194
